In [14]:
import pandas as pd

In [18]:
pivotted = pd.read_csv(r'C:\Users\achopra8\Documents\BDD Climate Control\2019f150VIN1FTEW1C53KFB01079_pivotted_finaltripflags.csv')
pivotted.head()

,cvdcus_timestamp_s_3,epoch,epoch_diff,trip,BrkTot_Tq_Actl,HvacBlwrFront_D_Stat,Veh_V_ActlEng,GearLvrPos_D_Actl,ApedPos_Pc_ActlArb,EngAout_N_Actl,...,CoolantFanStepAct,AirAmb_Te_ActlFilt,Outside_Air_Temp_Stat,OdometerMasterValue,Ignition_Status,VehLong2_A_Actl,RearPassWindowPos,DriverWindowPosition,PassWindowPosition,temp_gap
0,2019-05-01 14:40:01,1.556722e+09,NaN,1,742.666667,Off,0.0,Park,0.0,0.0,...,Step2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.5
1,2019-05-01 14:40:02,1.556722e+09,1.0,1,2094.000000,NaN,0.0,Park,0.0,0.0,...,Step2,19.0,10.0,2707.0,NaN,NaN,NaN,NaN,NaN,13.5
2,2019-05-01 14:40:03,1.556722e+09,1.0,1,2702.074074,NaN,0.0,Park,0.0,0.0,...,NaN,19.0,10.0,2707.0,Off,NaN,NaN,NaN,NaN,13.5
3,2019-05-01 14:40:04,1.556722e+09,1.0,1,2725.655172,NaN,0.0,Park,0.0,0.0,...,Step2,19.0,10.0,2707.0,Off,NaN,NaN,NaN,NaN,13.5
4,2019-05-01 14:40:05,1.556722e+09,1.0,1,2410.736842,NaN,0.0,Park,0.0,0.0,...,NaN,19.0,10.0,2707.0,NaN,NaN,NaN,NaN,NaN,13.5


In [224]:
from sklearn.impute import KNNImputer
def make_dummies_and_impute(trip):
    cols_to_ffill = ['BrkTot_Tq_Actl', 'Veh_V_ActlEng', 'GearLvrPos_D_Actl','ApedPos_Pc_ActlArb', 'EngAout_N_Actl', 'HvacEvap_Te_Actl', 'HvacEvap_Te_Rq', 'HvacAirCond_B_Rq', 'BattULo_U_Actl', 'AirAmb_Te_ActlFilt', 'Outside_Air_Temp_Stat', 'OdometerMasterValue', 'VehLong2_A_Actl', 'temp_gap']
    #trip = trip[['epoch'] + list(trip.columns[4:])]
    #trip = trip.drop(['Ignition_Status', 'RearPassWindowPos', 'DriverWindowPosition','PassWindowPosition'], axis=1) # ignition_status only needed to define trips; window position signals are bad quality so we will leave them out for now
    trip[cols_to_ffill] = trip[cols_to_ffill].fillna(method='ffill')
    imputer = KNNImputer(weights='distance')
    trip_one_hot_encoded = pd.get_dummies(trip)
    trip = pd.DataFrame(imputer.fit_transform(trip_one_hot_encoded), index=trip.index, columns=trip_one_hot_encoded.columns)
    return trip_one_hot_encoded

In [255]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
# function to plot gap between actual and requested temp.
def plot_vs_time(df, feature, temp_features_to_append, title='Features vs. Time', xlab='Time Elapsed (s)'):  
    fig, ax1 = plt.subplots(figsize=(30, 14))
    plt.title(title, fontsize=30)
    plt.xticks(fontsize=20)
    ax2 = ax1.twinx()
    df['epoch'] = (df['epoch'] - df['epoch'].min()).astype(int)
    df_for_vis = pd.melt(df, id_vars=['epoch'], value_vars=temp_features_to_append, var_name='Feature', value_name='temp')
    df_for_vis['time_elap'] = (df_for_vis['epoch'] - df_for_vis['epoch'].min()).astype(int)
    sns.lineplot(x="time_elap", y="temp", hue="Feature", data=df_for_vis, ax=ax1, marker='o', markersize=10, legend=False, palette={'HvacEvap_Te_Actl':'blue', 'temp_gap':'green', 'HvacEvap_Te_Rq':'orange'}) 
    sns.lineplot(x='epoch', y=feature, data=df, ax=ax2, marker='o', markersize=10, color='purple')
    ax1.set_ylabel('Temperature (C)', fontsize=20)
    ax1.tick_params(axis='y', which='major', labelsize=20)
    ax2.set_ylabel(feature, fontsize=20)
    ax2.tick_params(axis='y', which='major', labelsize=20)
    ax1.set_xlabel(xlab, fontsize=20)
    custom_lines = [Line2D([0], [0], lw=4, color='Blue'),
                Line2D([0], [0], lw=4, color='orange'),
                Line2D([0], [0], lw=4, color='green'), Line2D([0], [0], lw=4, color='Purple')]
    plt.legend(custom_lines, ['Actual', 'Requested', 'Gap', f'{feature}'], prop={'size': 20})

In [256]:
possible_cols = ['BrkTot_Tq_Actl',
 'Veh_V_ActlEng',
 'ApedPos_Pc_ActlArb',
 'EngAout_N_Actl',
 'BattULo_U_Actl',
 'AirCondFluidHi_P_Actl',
 'AirAmb_Te_ActlFilt',
 'Outside_Air_Temp_Stat',
 'OdometerMasterValue',
 'VehLong2_A_Actl',
 'HvacBlwrFront_D_Stat_15_Percent',
 'HvacBlwrFront_D_Stat_20_Percent',
 'HvacBlwrFront_D_Stat_25_Percent',
 'HvacBlwrFront_D_Stat_30_Percent',
 'HvacBlwrFront_D_Stat_35_Percent',
 'HvacBlwrFront_D_Stat_Off',
 "HvacBlwrFront_D_Stat_['10_Percent' '15_Percent' '5_Percent']",
 'GearLvrPos_D_Actl_Drive',
 'GearLvrPos_D_Actl_Park',
 'GearLvrPos_D_Actl_Reverse',
 "GearLvrPos_D_Actl_['Neutral' 'Reverse']",
 "GearLvrPos_D_Actl_['Park' 'Reverse']",
 'HvacAirCond_B_Rq_No',
 'HvacAirCond_B_Rq_Yes',
 "HvacAirCond_B_Rq_['No' 'Yes']",
 'CoolantFanStepAct_Step2',
 'CoolantFanStepAct_Step3',
 'CoolantFanStepAct_Step4',
 'CoolantFanStepAct_Step5',
 'CoolantFanStepAct_Step6',
 "CoolantFanStepAct_['Step2' 'Step3']",
 "CoolantFanStepAct_['Step2' 'Step4']",
 "CoolantFanStepAct_['Step5' 'Step6']",
 'HvacBlwrFront_D_Stat_45_Percent',
 'HvacBlwrFront_D_Stat_50_Percent',
 'HvacBlwrFront_D_Stat_55_Percent',
 'HvacBlwrFront_D_Stat_65_Percent',
 'HvacBlwrFront_D_Stat_70_Percent',
 'HvacBlwrFront_D_Stat_75_Percent',
 "HvacBlwrFront_D_Stat_['55_Percent' '65_Percent']",
 "HvacBlwrFront_D_Stat_['60_Percent' '65_Percent']",
 "HvacBlwrFront_D_Stat_['65_Percent' '70_Percent']",
 'CoolantFanStepAct_Step10',
 'CoolantFanStepAct_Step11',
 'CoolantFanStepAct_Step12',
 'CoolantFanStepAct_Step13',
 'CoolantFanStepAct_Step14',
 'CoolantFanStepAct_Step15',
 'CoolantFanStepAct_Step7',
 'CoolantFanStepAct_Step8',
 'CoolantFanStepAct_Step9',
 "CoolantFanStepAct_['Step10' 'Step11']",
 "CoolantFanStepAct_['Step13' 'Step14']",
 'HvacBlwrFront_D_Stat_60_Percent',
 'GearLvrPos_D_Actl_Neutral',
 'CoolantFanStepAct_Step1',
 "CoolantFanStepAct_['Step10' 'Step9']",
 "CoolantFanStepAct_['Step8' 'Step9']",
 'HvacBlwrFront_D_Stat_40_Percent',
 "HvacBlwrFront_D_Stat_['40_Percent' '45_Percent']",
 'CoolantFanStepAct_Step16',
 "CoolantFanStepAct_['Step4' 'Step6' 'Step9']",
 "CoolantFanStepAct_['Step4' 'Step6']",
 "CoolantFanStepAct_['Step6' 'Step7']",
 "HvacBlwrFront_D_Stat_['30_Percent' '35_Percent']",
 'HvacBlwrFront_D_Stat_10_Percent',
 "GearLvrPos_D_Actl_['Drive' 'Reverse']",
 'HvacBlwrFront_D_Stat_80_Percent',
 "HvacBlwrFront_D_Stat_['55_Percent' '60_Percent']",
 'HvacBlwrFront_D_Stat_5_Percent',
 'HvacBlwrFront_D_Stat_85_Percent',
 'HvacBlwrFront_D_Stat_90_Percent',
 "GearLvrPos_D_Actl_['Drive' 'Park']",
 "CoolantFanStepAct_['Step14' 'Step15']",
 'GearLvrPos_D_Actl_Sport_DriveSport',
 "CoolantFanStepAct_['Step7' 'Step8']",
 "HvacBlwrFront_D_Stat_['45_Percent' '50_Percent']",
 "HvacBlwrFront_D_Stat_['80_Percent' '85_Percent']",
 "HvacBlwrFront_D_Stat_['20_Percent' '25_Percent']",
 "HvacBlwrFront_D_Stat_['25_Percent' '30_Percent']",
 'CoolantFanStepAct_Fan_Cmd_Off',
 'CoolantFanStepAct_Step17',
 "HvacBlwrFront_D_Stat_['60_Percent' '65_Percent' '70_Percent' '75_Percent']",
 "HvacBlwrFront_D_Stat_['80_Percent' '85_Percent' '90_Percent']",
 "CoolantFanStepAct_['Step11' 'Step12']",
 'CoolantFanStepAct_Step18',
 "CoolantFanStepAct_['Step12' 'Step13']",
 "HvacBlwrFront_D_Stat_['65_Percent' '75_Percent']",
 "CoolantFanStepAct_['Step5' 'Step7']",
 "GearLvrPos_D_Actl_['Drive' 'Neutral' 'Park']",
 "CoolantFanStepAct_['Step15' 'Step16']",
 "HvacBlwrFront_D_Stat_['10_Percent' '15_Percent' '20_Percent']",
 "HvacBlwrFront_D_Stat_['40_Percent' '45_Percent' '50_Percent']",
 "CoolantFanStepAct_['Step3' 'Step4']",
 "CoolantFanStepAct_['Step4' 'Step5']",
 "HvacBlwrFront_D_Stat_['15_Percent' 'Off']",
 "HvacBlwrFront_D_Stat_['35_Percent' '40_Percent']",
 "HvacBlwrFront_D_Stat_['45_Percent' '50_Percent' '55_Percent']",
 "HvacBlwrFront_D_Stat_['60_Percent' '70_Percent']",
 "HvacBlwrFront_D_Stat_['70_Percent' '75_Percent']",
 "HvacBlwrFront_D_Stat_['75_Percent' '85_Percent']",
 "GearLvrPos_D_Actl_['Neutral' 'Park' 'Reverse' 'Sport_DriveSport']",
 "GearLvrPos_D_Actl_['Neutral' 'Park']",
 "CoolantFanStepAct_['Step1' 'Step2']",
 "CoolantFanStepAct_['Step7' 'Step9']",
 "GearLvrPos_D_Actl_['Drive' 'Neutral']",
 "HvacBlwrFront_D_Stat_['45_Percent' 'Off']",
 "HvacBlwrFront_D_Stat_['10_Percent' '5_Percent']",
 "GearLvrPos_D_Actl_['Drive' 'Neutral' 'Park' 'Reverse']",
 "HvacBlwrFront_D_Stat_['10_Percent' '20_Percent']",
 "HvacBlwrFront_D_Stat_['10_Percent' '15_Percent' '25_Percent']",
 "HvacBlwrFront_D_Stat_['20_Percent' '30_Percent']",
 "HvacBlwrFront_D_Stat_['35_Percent' 'Off']",
 "HvacBlwrFront_D_Stat_['10_Percent' '15_Percent']",
 "HvacBlwrFront_D_Stat_['15_Percent' '20_Percent']",
 "HvacBlwrFront_D_Stat_['50_Percent' '55_Percent']",
 "HvacBlwrFront_D_Stat_['70_Percent' '80_Percent']",
 "CoolantFanStepAct_['Step16' 'Step17']",
 "CoolantFanStepAct_['Step17' 'Step18']",
 "HvacBlwrFront_D_Stat_['15_Percent' '5_Percent']",
 "CoolantFanStepAct_['Step12' 'Step15']",
 "GearLvrPos_D_Actl_['Drive' 'Sport_DriveSport']",
 "CoolantFanStepAct_['Step10' 'Step12']",
 "CoolantFanStepAct_['Step12' 'Step16' 'Step8']",
 "CoolantFanStepAct_['Step5' 'Step7' 'Step8']",
 "GearLvrPos_D_Actl_['Drive' 'Neutral' 'Reverse']",
 "HvacBlwrFront_D_Stat_['10_Percent' '20_Percent' 'Off']",
 "CoolantFanStepAct_['Step11' 'Step8']",
 "HvacBlwrFront_D_Stat_['35_Percent' '45_Percent']",
 "HvacBlwrFront_D_Stat_['50_Percent' '55_Percent' '60_Percent']",
 "CoolantFanStepAct_['Fan_Cmd_Off' 'Step1']",
 "HvacBlwrFront_D_Stat_['30_Percent' 'Off']"]


In [266]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

@interact
def show_rel(impute=['Y', 'N'], trip_num=pivotted['trip'].unique(), feature=possible_cols, actl=widgets.Checkbox(value=True, description='Include Actual Temperature'), rq=widgets.Checkbox(value=True, description='Include Requested Temperature'), gap=widgets.Checkbox(value=True, description='Include Gap')):
    temp_features_to_plot = []
    if actl:
        temp_features_to_plot.append('HvacEvap_Te_Actl')
    if rq:
        temp_features_to_plot.append('HvacEvap_Te_Rq')
    if gap:
        temp_features_to_plot.append('temp_gap')
    trip = pivotted[(pivotted['trip'] == trip_num) & (pivotted['HvacAirCond_B_Rq'] == 'Yes')]
    trip = trip[['epoch'] + list(trip.columns[4:])]
    if impute == 'Y':
        trip = make_dummies_and_impute(trip)
    else:
        trip = pd.get_dummies(trip)
    try:
        plot_vs_time(trip, feature, temp_features_to_plot)   
    except ValueError:
        print('Feature not present in trip!')
        plt.close()

interactive(children=(Dropdown(description='impute', options=('Y', 'N'), value='Y'), Dropdown(description='tri…

In [265]:
pivotted[(pivotted['HvacAirCond_B_Rq'] == 'Yes')]

,cvdcus_timestamp_s_3,epoch,epoch_diff,trip,BrkTot_Tq_Actl,HvacBlwrFront_D_Stat,Veh_V_ActlEng,GearLvrPos_D_Actl,ApedPos_Pc_ActlArb,EngAout_N_Actl,...,CoolantFanStepAct,AirAmb_Te_ActlFilt,Outside_Air_Temp_Stat,OdometerMasterValue,Ignition_Status,VehLong2_A_Actl,RearPassWindowPos,DriverWindowPosition,PassWindowPosition,temp_gap
9,2019-05-01 14:40:10,1.556722e+09,1.0,1,2106.222222,Off,0.000,Park,0.0,1333.500000,...,Step2,NaN,19.0,2707.0,Run,0.191905,NaN,NaN,NaN,14.70000
10,2019-05-01 14:40:11,1.556722e+09,1.0,1,1943.636364,Off,0.000,Park,0.0,1319.142857,...,NaN,NaN,19.0,2707.0,Run,0.195294,NaN,NaN,NaN,15.00000
11,2019-05-01 14:40:12,1.556722e+09,1.0,1,1930.000000,NaN,0.000,Park,0.0,1312.444444,...,NaN,19.00,19.0,2707.0,NaN,0.200909,NaN,NaN,NaN,15.00000
12,2019-05-01 14:40:13,1.556722e+09,1.0,1,1896.888889,NaN,0.000,Park,0.0,1319.875000,...,NaN,19.00,19.0,2707.0,NaN,0.198095,NaN,NaN,NaN,14.96875
13,2019-05-01 14:40:14,1.556722e+09,1.0,1,1829.777778,NaN,0.000,Park,0.0,1307.285714,...,['Step2' 'Step4'],19.00,NaN,NaN,NaN,0.201000,NaN,NaN,NaN,14.93750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183198,2019-07-26 03:05:10,1.564110e+09,1.0,125,952.000000,NaN,0.215,Drive,0.0,522.750000,...,Step6,NaN,24.5,6872.0,Run,0.193333,NaN,NaN,NaN,0.75000
183199,2019-07-26 03:05:11,1.564110e+09,1.0,125,1151.000000,NaN,0.000,Reverse,0.0,523.469388,...,NaN,NaN,24.5,6872.0,Run,0.162222,NaN,NaN,NaN,0.87500
183202,2019-07-26 03:05:14,1.564110e+09,1.0,125,1916.000000,NaN,0.000,Park,0.0,580.716981,...,NaN,25.75,24.5,6872.0,NaN,0.165000,Undefined,Bet10Percent_60Percent,Fully_Closed,0.56250
183203,2019-07-26 03:05:15,1.564110e+09,1.0,125,1944.000000,NaN,0.000,Park,0.0,593.000000,...,Step6,25.75,NaN,NaN,Run,0.157143,NaN,NaN,NaN,0.50000
